In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.0 MB/s 
     |████████████████████████████████| 596 kB 88.4 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 6.6 MB 60.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
from transformers import AutoTokenizer, ElectraForSequenceClassification

In [4]:
# GPU 사용
device = torch.device("cuda")

In [8]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator",num_labels=3).to(device)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
path = '/content/drive/MyDrive/CP1/CP1_koelectra_3label/model_koelectra_label3_epoch50.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [10]:
def predict(text):
    encoded_review = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=128,
        padding=True,
        add_special_tokens=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output[0], dim=1)

    print('입력 텍스트: ' + text)
    if str(prediction) == "tensor([1], device='cuda:0')":
        print('감정: 긍정적')
    elif str(prediction) == "tensor([2], device='cuda:0')":
        print('감정: 부정적')
    else:
        print('감정: 중립')

In [11]:
predict('액션이 없는데도 재미 있는 몇안되는 영화')

입력 텍스트: 액션이 없는데도 재미 있는 몇안되는 영화
감정: 부정적


In [12]:
text = ['관종 김건희 방치하고 대통령실 꼴 보면 자업자득임',
'민좆에 병신 하나 더 들어가서 나쁠것 없지 ㅋㅋㅋㅋㅋ 그래봤자 이재명 꼬붕 하나 더 늘어날 뿐',
'이준석은 언론 어떻게 다루는지 잘 모르는거 같음.',
'윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다',
'이준석 아니었으면 국힘 지지도 나락갔을테고  이번 대선도 나락이었을텐데  꼭 이렇게 이준석을 힘들게 해야 함??   "당 대표 중심으로 협력해라!"  이 말 한마디 하기가 글케 어려움??   진짜 실망스럽다',
'실망스럽다',
'이준석 요즘 너무 잘하는거 같지 않음?',
'이준석 요즘 잘하는거 같음']

In [13]:
%%time
for i in text:
    print(predict(i))

입력 텍스트: 관종 김건희 방치하고 대통령실 꼴 보면 자업자득임
감정: 중립
None
입력 텍스트: 민좆에 병신 하나 더 들어가서 나쁠것 없지 ㅋㅋㅋㅋㅋ 그래봤자 이재명 꼬붕 하나 더 늘어날 뿐
감정: 긍정적
None
입력 텍스트: 이준석은 언론 어떻게 다루는지 잘 모르는거 같음.
감정: 중립
None
입력 텍스트: 윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다
감정: 중립
None
입력 텍스트: 이준석 아니었으면 국힘 지지도 나락갔을테고  이번 대선도 나락이었을텐데  꼭 이렇게 이준석을 힘들게 해야 함??   "당 대표 중심으로 협력해라!"  이 말 한마디 하기가 글케 어려움??   진짜 실망스럽다
감정: 부정적
None
입력 텍스트: 실망스럽다
감정: 부정적
None
입력 텍스트: 이준석 요즘 너무 잘하는거 같지 않음?
감정: 중립
None
입력 텍스트: 이준석 요즘 잘하는거 같음
감정: 중립
None
CPU times: user 83.9 ms, sys: 1.98 ms, total: 85.9 ms
Wall time: 88.6 ms
